##  Week 8.2: Regression 

*  Student Name: Abraham Abate
*  Instructor: Cary Jim
*  DSC 530: Data Exploration and Analysis
*  Data Science Dept., BU
*  Date: 07/27/2024



In [1]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

Let's load up the NSFG data again.

In [2]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [3]:
# import Libraries
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import pandas as pd

import nsfg

## 11.10 Exercises

**Exercise 11.1:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [4]:
# load the NSFG data
preg = nsfg.ReadFemPreg()
preg.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [5]:
# select columns
live = preg[preg.outcome == 1]
firsts = live[live.birthord == 1]
others = live[live.birthord != 1]

# select pregnancies longer than 30 weeks
live = live[live.prglngth>30]

live['isfirst'] = live.birthord == 1

In [6]:
# reads the respondent file
resp = nsfg.ReadFemResp()

# replace resp.index with resp.caseid.
resp.index = resp.caseid

# join live the “left” table, and passed resp, which is the “right” table.
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3332)

In [7]:
# check nbrnaliv values 
live["nbrnaliv"].value_counts()


1.0    8746
2.0     112
3.0      13
5.0       5
4.0       4
Name: nbrnaliv, dtype: int64

In [8]:
# count na values
live["nbrnaliv"].isna().sum()

4

In [9]:
# Drop na values from selected features
data = live.dropna(subset=['prglngth', 'birthord', 'race', 'nbrnaliv'])

In [10]:
# develop the first argument which is a formula string
formula = 'prglngth ~ birthord==1 + race == 2 + nbrnaliv>1'

# create and fit regression model
model = smf.ols(formula, data=data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.35
Date:                Sun, 28 Jul 2024   Prob (F-statistic):           4.56e-22
Time:                        08:44:50   Log-Likelihood:                -18240.
No. Observations:                8880   AIC:                         3.649e+04
Df Residuals:                    8876   BIC:                         3.652e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7611      0.039   1005.672      0.000      38.685      38.837
birthord == 1[T.True]     0.1024      0.040      2.548      0.011       0.024       0.181
race == 2[T.True]         0.1397      0.042      3.326      0.001       0.057       0.222
nbrnaliv > 1[T.True]     -1.4945      0.164     -9.085      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.243   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.552
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.708   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Interpretation:
- The result contains an intercept and three slopes, which estimate the average contribution of each predictor with the two predictors (birthord and race) held constant.

**Exercise 11.3:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [11]:
# reads the respondent file
resp = nsfg.ReadFemResp()

# replace resp.index with resp.caseid.
resp.index = resp.caseid

# join live the “left” table, and passed resp, which is the “right” table.
joined_df = live.join(resp, on='caseid', rsuffix='_r')
joined_df.shape

(8884, 3332)

In [12]:
# check the newly created df
joined_df.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167


In [13]:
# Drop na values from the dependent variable
data = joined_df.dropna(subset=['numbabes'])

In [14]:
# nonlinear model of age
# create a new variable using quadratic term 
data['age2'] = data.age_r**2

In [15]:
# check the new column
data['age2']

0        1936
1        1936
2         400
3         400
4         400
         ... 
13581    1225
13584     961
13588    1369
13591    1369
13592    1369
Name: age2, Length: 8884, dtype: int64

In [16]:
# develop the first argument which is a formula string
formula = 'numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
formula = 'numbabes ~ age_r + age2 + C(race) + totincr + educat'

# create and fit regression model 
model = smf.poisson(formula, data=data)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.677002
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Sun, 28 Jul 2024   Pseudo R-squ.:                 0.03686
Time:                        08:45:05   Log-Likelihood:                -14898.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                3.681e-243
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0324      0.169     -6.098      0.000      -1.364      -0.701
C(race)[T.2]    -0.1401      0.015     -9.479      0.000      -0.169      -0.111
C(race)[T.3]    -0.0991      0.025     -4.029      0.000      -0.147      -0.051
age_r            0.1556      0.010     15.006      0.000       0.135       0.176
age2            -0.0020      0.000    -13.102      0.000      -0.002      -0.002
totincr         -0.0187      0.002     -9.830      0.000      -0.022      -0.015
educat          -0.0471      0.003    -16.076      0.000      -0.053      -0.041
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [17]:
# create a DataFrame with the desired feature
columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
df = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
df

,age_r,age2,age3,race,totincr,educat
0,35,1225,42875,1,14,16


In [18]:
# generate predictions 
results.predict(df)

0    2.496802
dtype: float64

Interpretation: A woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000 will have 3 children. 

**Exercise 11.4:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [19]:
# Drop na values from the dependent variable
data = joined_df.dropna(subset=['rmarital'])

In [20]:
# The responses column and its values count.
data['rmarital'].value_counts()

1    5027
6    1403
2     914
4     860
5     575
3     105
Name: rmarital, dtype: int64

Value Label:
- 1 -- CURRENTLY MARRIED
- 2 -- NOT MARRIED BUT LIVING WITH OPP SEX PARTNER 732
- 3 -- WIDOWED
- 4 -- DIVORCED
- 5 -- SEPARATED FOR REASONS OF MARITAL DISCORD 279
- 6 -- NEVER BEEN MARRIED

Note: These are the features we need for the regression model
* Dependent variable == marital status (rmartial)
* Independent variables == age, race, Total income and education level

In [21]:
# nonlinear model of age
# create a new variable using quadratic term 
data['age2'] = data.age_r**2

In [22]:
# develop the first argument which is a formula string
formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'

# create and fit regression model 
model = smf.mnlogit(formula, data=data)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Sun, 28 Jul 2024   Pseudo R-squ.:                  0.1682
Time:                        08:45:06   Log-Likelihood:                -9630.7
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0156      0.805     11.199      0.000       7.438      10.593
C(race)[T.2]    -0.9237      0.089    -10.418      0.000      -1.097      -0.750
C(race)[T.3]    -0.6179      0.136     -4.536      0.000      -0.885      -0.351
age_r           -0.3635      0.051     -7.150      0.000      -0.463      -0.264
age2             0.0048      0.001      6.103      0.000       0.003       0.006
totincr         -0.1310      0.012    -11.337      0.000      -0.154      -0.108
educat          -0.1953      0.019    -10.424      0.000      -0.232      -0.159
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9570      3.020      0.979      0.328      -2.963       8.877
C(race)[T.2]    -0.4411      0.237     -1.863      0.062      -0.905       0.023
C(race)[T.3]     0.0591      0.336      0.176      0.860      -0.600       0.718
age_r           -0.3177      0.177     -1.798      0.072      -0.664       0.029
age2             0.0064      0.003      2.528      0.011       0.001       0.011
totincr         -0.3258      0.032    -10.175      0.000      -0.389      -0.263
educat          -0.0991      0.048     -2.050      0.040      -0.194      -0.004
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.5238      1.205     -2.924      0.003      -5.886      -1.162
C(race)[T.2]    -0.3213      0.093     -3.445      0.001      -0.504      -0.139
C(race)[T.3]    -0.7706      0.171     -4.509      0.000      -1.106      -0.436
age_r            0.1155      0.071      1.626      0.104      -0.024       0.255
age2            -0.0007      0.001     -0.701      0.483      -0.003       0.001
totincr         -0.2276      0.012    -19.621      0.000      -0.250      -0.205
educat           0.0667      0.017      3.995      0.000       0.034       0.099
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8963      1.305     -2.220      0.026      -5.453      -0.339
C(race)[T.2]    -1.0407      0.104    -10.038      0.000      -1.244      -0.837
C(race)[T.3]    -0.5661      0.156     -3.635      0.000      -0.871      -0.261
age_r            0.2411      0.079      3.038      0.002       0.086       0.397
age2            -0.0035      0.001     -2.977      0.003      -0.006      -0.001
totincr         -0.2932      0.015    -20.159      0.000      -0.322      -0.265
educat          -0.0174      0.021     -0.813      0.416      -0.059       0.025
-

In [23]:
results.params

,0,1,2,3,4
Intercept,9.015602,2.956983,-3.523815,-2.896280,8.053331
C(race)[T.2],-0.923692,-0.441060,-0.321297,-1.040679,-2.187093
C(race)[T.3],-0.617889,0.059145,-0.770606,-0.566105,-1.961066
age_r,-0.363496,-0.317738,0.115476,0.241079,-0.212661
age2,0.004799,0.006444,-0.000720,-0.003533,0.001865
totincr,-0.131014,-0.325819,-0.227557,-0.293237,-0.294451
educat,-0.195287,-0.099074,0.066676,-0.017403,-0.074182


Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [24]:
# create a DataFrame with the desired feature
columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
df = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
df

,age_r,age2,race,totincr,educat
0,25,625,2,11,12


In [25]:
# generate predictions 
results.predict(df)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122


Intepretation: 
- There s a 75% chance of the woman being married and a 13% chance of just living with a man, a 0.1% chance of being widowed, a 3.5% chance of being divorced, a 2.4% chance of being separated and a 6.6% chance of never been married.

In [26]:
# Solution

# This person has a 75% chance of being currently married,
# a 13% chance of being "not married but living with opposite
# sex partner", etc.

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122
